# Evaluate Similarity Grouping

In this notebook, we evaluate how effective a relation can be integrated using the NoiseAwareGroupBy Operator.
Therefore, we utilize the [Music Brainz 20K](https://dbs.uni-leipzig.de/research/projects/benchmark-datasets-for-entity-resolution).

The dataset contains modified (usign the DAPO data generator) song records from different sources.
The goal is to group same songs into buckets. E.g. The records {"title": "Daniel Balavoine - L'enfant aux yeux d'Italie", "artist": null, "album": "De vous à elle en passant par moi", ...} and {"name": L'enfant aux yeux d'Italie - De vous à elle en passant par moi", "artist": "Daniel Balavoine", "album": null} describe the same song.

The column "CID" describes the cluster of the record. Using the  `SoftAggregateScikit` operator, we determine clusters and calculate the metrics:
* Adjusted Rand Index (ARI)
* Normalized Mutual Information (NMI)
* Fowlkes-Mallows Index (FMI)


In [1]:
import pandas as pd
from models import ModelMgr
from models.embedding.SentenceTransformer import SentenceTransformerEmbeddingModel

from db.operators import Dummy, SoftAggregateScikit
from db.operators.Aggregate import SetAggregation
from sklearn.cluster import KMeans, DBSCAN, HDBSCAN

from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, fowlkes_mallows_score

In [2]:
m = ModelMgr()
stem = SentenceTransformerEmbeddingModel(m)

drop_na = True

In [3]:
df_music = pd.read_csv("../data/musicbrainz-20-A01.csv", index_col=0).drop(columns=["length"], axis=1)
significant_cols = ["title", "artist", "album", "year", "language"]
df_music.head()

,CID,CTID,SourceID,id,number,title,artist,album,year,language
TID,,,,,,,,,,
1,1,1,2,MBox7368722-HH,9,Daniel Balavoine - L'enfant aux yeux d'Italie,NaN,De vous à elle en passant par moi,75,French
2,2512,5,4,139137-A047,7,007,[unknown],Cantigas de roda (unknown),NaN,Por.
3,2,1,2,MBox38440522-HH,17,Action PAINTING! - Mustard Gas,NaN,There and Back Again Lane,95,English
4,3,1,5,4489993,10,Your Grace,Kathy Troccoli,Comfort,2005,English
5,4,1,5,10339621,2,Well You Needn't,Ernie Stadler Jazz Quintet,First Down,2010,English


In [4]:
def evaluate(df, cluster_columns, id_column, cluster_class, cluster_params, serialization_mode, reduce_dimensions, drop_na):
    key = (str(cluster_class), str(serialization_mode), str(reduce_dimensions), str(drop_na))

    if drop_na:
        df = df.dropna()

    if cluster_class == KMeans:
        cluster_params = {"n_clusters": len(df["CID"].unique())}

    columns = [col.strip() for col in df.columns]
    data = [[str(y) for y in x] for x in df.itertuples(name=None)]

    d = Dummy("data", ["tid"] + columns, data).open()
    agg = SoftAggregateScikit(
        d,
        cluster_columns,
        [SetAggregation("tid", "ids")],
        em=stem,
        cluster_class = cluster_class,
        cluster_params = cluster_params,
        serialization_mode = serialization_mode,
        reduce_dimensions=reduce_dimensions
    )

    predictions = []
    for i, row in enumerate(agg.open()):
        predictions.append(pd.Series([i for _ in range(len(row["ids"]))], index=[int(idx) for idx in row["ids"]]))

    predicted_labels = pd.concat(predictions).sort_index()
    true_labels = df[id_column]

    ari = adjusted_rand_score(true_labels, predicted_labels)
    nmi = normalized_mutual_info_score(true_labels, predicted_labels)
    fmi = fowlkes_mallows_score(true_labels, predicted_labels)

    result = {"adjusted_rand_score": ari, "normalized_mutual_info_score": nmi, "fowlkes_mallows_score": fmi}

    print(key, result)

    return key, result

overall_results = {}

In [5]:
cluster_classes = [
    (KMeans, None),
    (DBSCAN, {"eps": 0.1, "min_samples": 1}),
    (HDBSCAN, {"min_cluster_size": 2}),
]

for cc, cp in cluster_classes:
    for sm in ["FIELD_SERIALIZED", "FULL_SERIALIZED"]:
        for dn in [True, False]:
            for dim in [2, 10, 50, 100, None]:
                res = evaluate(df_music, significant_cols, "CID", cluster_class=cc, cluster_params=cp, serialization_mode = sm, reduce_dimensions = dim, drop_na = dn)
                overall_results[res[0]] = res[1]

/home/nico/PycharmProjects/Master/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


("<class 'sklearn.cluster._kmeans.KMeans'>", 'FIELD_SERIALIZED', '2', 'True') {'adjusted_rand_score': -2.4531393179505998e-05, 'normalized_mutual_info_score': 0.9940243916923467, 'fowlkes_mallows_score': 0.0}


/home/nico/PycharmProjects/Master/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


("<class 'sklearn.cluster._kmeans.KMeans'>", 'FIELD_SERIALIZED', '10', 'True') {'adjusted_rand_score': 0.023228762452172664, 'normalized_mutual_info_score': 0.9938931018904221, 'fowlkes_mallows_score': 0.023911404992940522}


/home/nico/PycharmProjects/Master/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

("<class 'sklearn.cluster._hdbscan.hdbscan.HDBSCAN'>", 'FULL_SERIALIZED', '10', 'False') {'adjusted_rand_score': 0.5204453748525378, 'normalized_mutual_info_score': 0.956562231167877, 'fowlkes_mallows_score': 0.5305845184758278}


/home/nico/PycharmProjects/Master/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


("<class 'sklearn.cluster._hdbscan.hdbscan.HDBSCAN'>", 'FULL_SERIALIZED', '50', 'False') {'adjusted_rand_score': 0.5181536880148707, 'normalized_mutual_info_score': 0.9566549887859075, 'fowlkes_mallows_score': 0.528531015577852}


/home/nico/PycharmProjects/Master/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


("<class 'sklearn.cluster._hdbscan.hdbscan.HDBSCAN'>", 'FULL_SERIALIZED', '100', 'False') {'adjusted_rand_score': 0.527119186440687, 'normalized_mutual_info_score': 0.9572422943253446, 'fowlkes_mallows_score': 0.5362443961617454}
("<class 'sklearn.cluster._hdbscan.hdbscan.HDBSCAN'>", 'FULL_SERIALIZED', 'None', 'False') {'adjusted_rand_score': 0.871507546722749, 'normalized_mutual_info_score': 0.9923116420069015, 'fowlkes_mallows_score': 0.8731580103378349}


In [29]:
keys = ["cluster", "serialization", "dimension", "drop_na"]
evaluation_results_list = [v | {ki: vi for ki, vi in zip(keys, k)} for k, v in overall_results.items()]
df_evaluation_results = pd.DataFrame.from_records(evaluation_results_list)
df_evaluation_results["cluster"] = df_evaluation_results["cluster"].apply(lambda x: x.split(".")[-1].replace("'>", ""))
df_evaluation_results = df_evaluation_results.set_index(keys)
df_evaluation_results.head()

adjusted_rand_score  \
cluster serialization    dimension drop_na                        
KMeans  FIELD_SERIALIZED 2         True               -0.000025   
                         10        True                0.023229   
                         50        True                0.032896   
                         100       True                0.030507   
                         None      True                0.738910   

                                            normalized_mutual_info_score  \
cluster serialization    dimension drop_na                                 
KMeans  FIELD_SERIALIZED 2         True                         0.994024   
                         10        True                         0.993893   
                         50        True                         0.994037   
                         100       True                         0.993958   
                         None      True                         0.998575   

                                            fowlkes_mallows_score  
cluster serialization    dimension drop_na                         
KMeans  FIELD_SERIALIZED 2         True                  0.000000  
                         10        True                  0.023911  
                         50        True                  0.033501  
                         100       True                  0.031488  
                         None      True                  0.739140

In [32]:
df_evaluation_results.to_csv("results/EvaluateClustering.csv")